In [2]:
from langchain.document_loaders import PyPDFLoader
import API_KEYS

# PDF 파일을 로드
documents = PyPDFLoader('seoul.pdf').load()


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서를 청크로 분할
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

# 분할된 문서를 docs 변수에 저장
docs = split_docs(documents)

In [4]:
 #OpenAI 임베딩 모델 사용
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=API_KEYS.OPENAI_API_KEY)

# Chroma에 벡터 저장, 저장 장소는 현재 디렉토리로 설정
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory=".")

# ChatOpenAI를 통해 LLM 생성
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"  # GPT-3.5-turbo 모델 사용
llm = ChatOpenAI(model_name=model_name, api_key=API_KEYS.OPENAI_API_KEY)

/var/folders/7s/07zh116j7lj43gqgztwcw8xm0000gn/T/ipykernel_68525/1696850331.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name=model_name, api_key=API_KEYS.OPENAI_API_KEY)


In [5]:
# Q&A 체인 로드
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

/var/folders/7s/07zh116j7lj43gqgztwcw8xm0000gn/T/ipykernel_68525/1769498800.py:3: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff", verbose=True)


In [6]:
# 쿼리 작성 후 유사성 검색으로 답변 생성

user_preferences = {
            "culturalTourism": 0.8,
            "historicalTourism": 0.5,
            "shopping": 0.3,
            "leisureSports": 0.5,
            "restaurant": 0.1,
            "natureTourism": 0.4
}

user_position = (37.552987017, 126.972591728)



query = (f"너는 나의 여행 플래너야. 아래 문항에 대해 확실하지 않은 정보가 있다면 주지마."
         f"나는 문화관광에 {user_preferences['culturalTourism']}만큼,"
         f"역사관광에 {user_preferences['historicalTourism']}만큼,"
         f"쇼핑에 {user_preferences['shopping']}만큼,"
         f"레저스포츠와 체험에 {user_preferences['leisureSports']}만큼,"
         f"음식에 {user_preferences['restaurant']} 만큼, 그리고"
         f"자연관광에 {user_preferences['natureTourism']} 만큼 관심이 있어" 
         f"현재 위치는 ({user_position[0]},{user_position[1]})인데, 가까이 있는 어딜 가보면 좋을까? 5개의 항목을 추천해주고 이유도 같이 설명해줘.")

matching_docs = db.similarity_search(query)
answer = chain.run(input_documents=matching_docs, question=query)

matching_docs = db.similarity_search(query)
answer = chain.run(input_documents=matching_docs, question=query)

print(answer)

/var/folders/7s/07zh116j7lj43gqgztwcw8xm0000gn/T/ipykernel_68525/2043794333.py:26: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer = chain.run(input_documents=matching_docs, question=query)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
색다른 하루
서울에서 여행을 좀 더 특별하게 즐기고 싶다면 이곳에서만 
경험할 수 있는 체험을 해보는 건 어떨까. 도심 속 등산부터 
템플스테이까지 서울에서 만나볼 수 있다.
봉은사 템플스테이
 서울 강남구 봉은사로 531 (9호선 봉은사역 1번 출구 인근)
 상월선원 템플스테이 100,000원, 쉼 템플스테이 80,000원
 02-3218-4826    temple.bongeunsa.org
조선 시대 서울에서 가장 큰 사찰이었던 봉은사에서 
불교문화를 경험할 수 있다. 금요일 오후 7시에 
시작하는 상월선원 템플스테이(2박 3일)와 쉼 
템플스테이(1박 2일)가 있다.
서울 도심 등산관광센터
 서울시 강북구 삼양로 173길 52(우이신설선 북한산우이역 
2번 출구 인근)    09:00-18:00, 월요일, 설날/추석 당일 휴관
 1533-2608    seoulhiking.or.kr
뛰어난 자연경관을 지닌 서울은 도시와 자연의 매력이 
공존한다. 서울 도심 등산관광센터는 우이신설선 
‘북한산우이역’에서 도보 5분 거리에 위치해 있으며, 
등산 관광객들을 위한 다양한 서비스를 제공한다.
 강남코엑스점 서울 강남구 테헤란로87길 58 (9호선 
삼성중앙역 4번 출구 인근) 서울드래곤시티점 서울 용산구 
청파로20길 95(1호선·경의중앙선 용산역 3번 출구 인근)    
 24시간 운영    7luck.com
세븐럭 카지노
서울 강남의 중심과 용산에 